## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-18-01-11-24 +0000,nyt,"For Hegseth, There Is One Boat Strike He Doesn...",https://www.nytimes.com/2025/12/17/us/politics...
1,2025-12-18-01-09-49 +0000,nyt,Trump’s National Address: What Time and What t...,https://www.nytimes.com/2025/12/17/us/politics...
2,2025-12-18-01-07-30 +0000,wapo,Trump to address the nation amid economic pres...,https://www.washingtonpost.com/politics/2025/1...
3,2025-12-18-01-04-06 +0000,nyt,Trump Unveils Presidential ‘Walk of Fame’ With...,https://www.nytimes.com/2025/12/17/us/politics...
4,2025-12-18-01-04-00 +0000,wsj,U.S. Accepts Fault for Reagan Airport Collisio...,https://www.wsj.com/business/airlines/u-s-acce...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2393/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,56
140,reiner,25
222,new,22
724,bondi,20
20,venezuela,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
126,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...,146
171,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...,142
332,2025-12-17-11-00-00 +0000,latimes,Arellano: With his un-Christian Rob Reiner ins...,https://www.latimes.com/california/story/2025-...,114
398,2025-12-17-02-58-00 +0000,wsj,Trump Orders Blockade of Sanctioned Oil Tanker...,https://www.wsj.com/world/americas/trump-order...,113
355,2025-12-17-09-44-00 +0000,wsj,Oil Prices Climb as Trump’s Venezuela Blockade...,https://www.wsj.com/business/energy-oil/oil-pr...,112


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
126,146,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...
23,110,2025-12-18-00-39-21 +0000,nypost,Nick Reiner was ‘declining’ before parents’ mu...,https://nypost.com/2025/12/17/us-news/nick-rei...
413,73,2025-12-17-01-24-28 +0000,nypost,Famed Australian Rabbi Eli Schlanger killed in...,https://nypost.com/2025/12/16/world-news/famed...
392,71,2025-12-17-03-40-00 +0000,wsj,The Senate majority leader said Congress won’t...,https://www.wsj.com/politics/policy/house-gop-...
296,64,2025-12-17-12-59-17 +0000,nypost,Warner Bros. Discovery board urges shareholder...,https://nypost.com/2025/12/17/business/warner-...
398,57,2025-12-17-02-58-00 +0000,wsj,Trump Orders Blockade of Sanctioned Oil Tanker...,https://www.wsj.com/world/americas/trump-order...
171,57,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...
410,47,2025-12-17-02-00-00 +0000,wsj,Tuesday’s jobs report was the latest ominous s...,https://www.wsj.com/economy/jobs/white-collar-...
48,45,2025-12-18-00-03-57 +0000,nypost,LA Mayor Karen Bass’ ‘continuing negligence’ t...,https://nypost.com/2025/12/17/us-news/la-mayor...
68,45,2025-12-17-23-44-39 +0000,nypost,Judge scolds serial NYC dine-and-dash diva Pei...,https://nypost.com/2025/12/17/us-news/judge-sc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
